<a href="https://colab.research.google.com/github/ptmhoang97/train_dataset/blob/main/TransferLearning_GGdrive_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
batch_size_pipeline = 64
num_steps_training = 10000
learning_rate_pipeline = 0.1
warmup_learning_rate_pipeline = 0.01
models_drive_path = "/content/drive/MyDrive/workspaces_tensorflow/test_resume_training"
model_name = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
train_dataset = "/content/drive/MyDrive/coco2017_train_car.zip"
test_dataset = "/content/drive/MyDrive/coco2017_validation_car.zip"

# 0. Setup Paths

In [ ]:
import os

In [ ]:
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc'),
    'MODEL_PATH': os.path.join(models_drive_path),
    'CHECKPOINT_PATH': os.path.join(models_drive_path, 'next_trained_model'), 
    'OUTPUT_PATH': os.path.join(models_drive_path, 'next_trained_model', 'export'), 
    'TFJS_PATH':os.path.join(models_drive_path, 'next_trained_model', 'tfjsexport'), 
    'TFLITE_PATH':os.path.join(models_drive_path, 'next_trained_model', 'tfliteexport'), 
    'PRETRAINED_MODEL_PATH':os.path.join(models_drive_path, model_name), 
 }

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join(models_drive_path, 'next_trained_model', 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME),
    'PIPELINE_CONFIG_BASE':os.path.join(models_drive_path, model_name, 'pipeline.config'),
    'CHECKPOINT_BASE':os.path.join(models_drive_path, model_name, 'checkpoint', 'ckpt-0'),
}

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}
        print(path)

# 1. Install TF Object Dectection API

In [ ]:
# https://www.tensorflow.org/install/source_windows

In [ ]:
if os.name=='nt':
    !pip install wget
    import wget

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

In [ ]:
print(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py'))

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [ ]:
import object_detection

# 2. Download dataset

In [ ]:
!unzip {train_dataset} -d {paths['IMAGE_PATH']}

In [ ]:
!unzip {test_dataset} -d {paths['IMAGE_PATH']}

# 3. Create Label Map

In [ ]:
labels = [{'name':'car', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 4. Create TF records

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

In [ ]:
train_dataset_folder = train_dataset[train_dataset.rfind("/")+1:train_dataset.rfind(".")]
test_dataset_folder = test_dataset[test_dataset.rfind("/")+1:test_dataset.rfind(".")]
print(train_dataset_folder)
print(test_dataset_folder)

In [ ]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], train_dataset_folder)} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], test_dataset_folder)} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

# 5. Copy Model Config to Training Folder

In [ ]:
if os.name =='posix':
    !cp {os.path.join(files['PIPELINE_CONFIG_BASE'])} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(files['PIPELINE_CONFIG_BASE'])} {os.path.join(paths['CHECKPOINT_PATH'])}

# 6. Update Config For Transfer Learning

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = batch_size_pipeline
pipeline_config.train_config.num_steps = num_steps_training
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.total_steps = num_steps_training
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.learning_rate_base = learning_rate_pipeline
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.warmup_learning_rate = warmup_learning_rate_pipeline
pipeline_config.train_config.fine_tune_checkpoint = files['CHECKPOINT_BASE']
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 7. Train the model

In [ ]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [ ]:
print(command)

In [ ]:
!{command}

# 10. Zip model and copy to GG Drive

In [ ]:
%cd {paths['MODEL_PATH']}

In [ ]:
export_model_path = "attempt_{}_{}".format(start_date_short,new_num)
export_model_zip = "{}.zip".format(export_model_path)
print(export_model_path)
print(export_model_zip)

In [ ]:
!zip -r {export_model_zip} {export_model_path}

In [ ]:
%cd "/content"

# 11. Notification via Email

In [ ]:
import datetime
import pytz

finish_date = str(datetime.datetime.now(pytz.timezone('Asia/Bangkok')))[:19]
finish_date_short = shorten_time(finish_date)

print(finish_date)
print(finish_date_short)

In [ ]:
sd_year = int(start_date_short[0:4])
sd_month = int(start_date_short[4:6])
sd_day = int(start_date_short[6:8])
sd_hour = int(start_date_short[9:11])
sd_min = int(start_date_short[11:13])
sd_sec = int(start_date_short[13:15])

fd_year = int(finish_date_short[0:4])
fd_month = int(finish_date_short[4:6])
fd_day = int(finish_date_short[6:8])
fd_hour = int(finish_date_short[9:11])
fd_min = int(finish_date_short[11:13])
fd_sec = int(finish_date_short[13:15])

In [ ]:
start = datetime.datetime(year=sd_year, month=sd_month, day=sd_day, hour=sd_hour, minute=sd_min,second=sd_sec)
end = datetime.datetime(year=fd_year, month=fd_month, day=fd_day, hour=fd_hour, minute=fd_min,second=fd_sec)
diff = end - start
print(diff)

In [ ]:
import smtplib
from email.message import EmailMessage

msg_subject = "[GG Colab] Model {}".format(start_date_short)
msg_content = "It take {} for training.".format(diff)

msg = EmailMessage()
msg['Subject'] = msg_subject
msg['From'] = '*'
msg['To']='ptmhoang97@gmail.com'
msg.set_content(msg_content)

server = smtplib.SMTP_SSL('smtp.gmail.com',465)
server.login("ptmhoang97@gmail.com","euclxykrqkkrquqz")
server.send_message(msg)
server.quit()